In [2]:
# !pip install peft

  ERROR: Could not find a version that satisfies the requirement peft (from versions: none)
ERROR: No matching distribution found for peft


In [1]:
from peft import PeftModel, PeftConfig
from transformers import LlamaForSequenceClassification, LlamaConfig, LlamaTokenizer,AutoConfig
import torch

ModuleNotFoundError: No module named 'peft'

In [2]:
ckpt_dir="/rds/general/user/l22/home/git_repos/ClinicalTransformerRelationExtraction/llama2_final_models_slurm/ckpt_0/"
pretrained_model="meta-llama/Llama-2-7b-hf"
mycache_dir="/rds/general/user/l22/home/llama_2/"

In [4]:
EN1_START = "[s1]"
EN1_END = "[e1]"
EN2_START = "[s2]"
EN2_END = "[e2]"
# keep the seq order
SPEC_TAGS = [EN1_START, EN1_END, EN2_START, EN2_END]

In [23]:
pre_config=LlamaConfig.from_pretrained(pretrained_model, num_labels=8)
peft_config=PeftConfig.from_pretrained(ckpt_dir, Tas)

In [24]:
llamaModel = LlamaForSequenceClassification.from_pretrained(
                pretrained_model,
                cache_dir=mycache_dir,
                config=pre_config,
    #             output_attentions=False,
    #             output_hidden_states=False,
                torch_dtype=getattr(torch, 'bfloat16'),
                low_cpu_mem_usage=True,  
            )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at meta-llama/Llama-2-7b-hf were not used when initializing LlamaForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing LlamaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LlamaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
tokenizer = LlamaTokenizer.from_pretrained(ckpt_dir)
pre_tokenizer = LlamaTokenizer.from_pretrained(pretrained_model)


In [27]:
tokenizer

LlamaTokenizer(name_or_path='/rds/general/user/l22/home/git_repos/ClinicalTransformerRelationExtraction/llama2_final_models_slurm/ckpt_0/', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'pad_token': '</s>'}, clean_up_tokenization_spaces=False)

In [28]:
last_token_idx = len(tokenizer)
tokenizer.add_tokens(SPEC_TAGS)
spec_token_new_ids = tuple([(last_token_idx + idx) for idx in range(len(tokenizer) - last_token_idx)])
total_token_num = len(tokenizer)

In [29]:
print(tokenizer.vocab_size)
print(pre_tokenizer.vocab_size)

32000
32000


In [30]:
# config = PeftConfig.from_pretrained(ckpt_dir)
# config.task_type="SEQ_CLS"
# config

In [31]:
llamaModel.resize_token_embeddings(total_token_num)

Embedding(32004, 4096)

In [32]:
model = PeftModel.from_pretrained(llamaModel,ckpt_dir)